You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Artificial Intelligence

1. Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence:
- Latest Trends:
  - Advancements in deep learning algorithms
  - Increased use of AI in healthcare and finance industries
  - Ethical considerations in AI development and deployment
- Key Players:
  - Google DeepMind
  - IBM Watson
  - Amazon Web Services
- Noteworthy News:
  - Breakthroughs in natural language processing
  - Implementation of AI in autonomous vehicles

2. Target Audience Anal

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has transitioned from a futuristic concept to a present-day reality that is reshaping numerous industries. Defined as the simulation of human intelligence processes by machines, AI plays a pivotal role in today's society. From enabling virtual assistants to powering autonomous vehicles, the influence of AI is undeniable. This blog post delves into the latest trends, key players, and significant news within the realm of Artificial Intelligence, catering to both business professionals and tech enthusiasts.

## Latest Trends in AI
The landscape of AI is constantly evolving, with advancements in deep learning algorithms leading the way. Progress in speech recognition and image processing is transforming the functionality of AI systems. Additionally, the growing integration of AI in healthcare and finance sectors is revolutionizing pat

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has transitioned from a futuristic concept to a present-day reality that is reshaping numerous industries. Defined as the simulation of human intelligence processes by machines, AI plays a pivotal role in today's society. From enabling virtual assistants to powering autonomous vehicles, the influence of AI is undeniable. This blog post delves into the latest trends, key players, and significant news within the realm of Artificial Intelligence, catering to both business professionals and tech enthusiasts.

## Latest Trends in AI
The landscape of AI is constantly evolving, with advancements in deep learning algorithms leading the way. Progress in speech recognition and image processing is transforming the functionality of AI systems. Additionally, the growing integration of AI in healthcare and finance sectors is revolutionizing patient care and financial services. However, as AI becomes more ubiquitous, ethical considerations in its development and implementation are gaining prominence.

## Key Players in the AI Industry
Numerous companies are driving innovation in the AI sector, shaping its trajectory. Google DeepMind, renowned for its pioneering work in deep learning, is pushing the boundaries of AI research. IBM Watson, equipped with cognitive computing capabilities, is making significant strides in sectors such as healthcare and finance. Amazon Web Services also stands out as a key player, providing AI services to businesses seeking to incorporate AI into their operations.

## Noteworthy News in AI Development
Recent advancements in natural language processing have simplified the interaction between machines and human language. Furthermore, the integration of AI in autonomous vehicles is paving the way for safer and more efficient transportation systems. These breakthroughs underscore AI's potential to enhance various facets of our lives and society as a whole.

## Ethical Considerations in AI
As AI technologies progress, ethical considerations surrounding their development and deployment are of utmost importance. The presence of bias in AI algorithms, concerns regarding data privacy, and the potential impact of AI on job displacement necessitate thoughtful deliberation. It is imperative for companies and researchers to address these ethical challenges to ensure that AI contributes positively to society.

## Conclusion
In summary, Artificial Intelligence presents a dynamic landscape with vast potential for growth and innovation. By staying abreast of the latest trends, key players, and significant news in AI, business professionals and tech enthusiasts can gain insight into the impact of AI across various industries. As AI continues to evolve, it is crucial to consider the ethical implications of its advancement and deployment. Embracing AI responsibly enables us to harness its transformative power for the betterment of the world.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Topic: The Impact of Social Media on Mental Health

1. Latest Trends, Key Players, and Noteworthy News:
- Latest Trends: The rise of mental health awareness on social media platforms, increased scrutiny on social media companies for their impact on mental health, and the use of social media for mental health advocacy.
- Key Players: Mental health professionals, social media platforms, mental health advocates, and researchers studying the link between social media and mental health.
- Noteworthy News: 

I now can give a great answer

Final Answer:
# The Impact of Social Media on Mental Health

## Introduction
In today's digital age, social media plays a significant role in shaping our daily lives, influencing our thoughts, behaviors, and emotions. While social media platforms offer unprecedented connectivity and opportunities for self-expression, there is a growing concern about their impact on mental health. The rise of mental health awareness on social media platforms has shed light on the importance of addressing the negative effects of excessive social media use. This blog post will delve into the complex relationship between social media and mental health, exploring both the positive and negative aspects while providing tips for promoting mental well-being in the digital age.

## The Positive and Negative Effects of Social Media on Mental Health
On one hand, social media has the power to connect individuals from across the globe, provide a platform for self-expression, and raise 

In [14]:
Markdown(result)

# The Impact of Social Media on Mental Health

## Introduction
In today's digital age, social media plays a significant role in shaping our daily lives, influencing our thoughts, behaviors, and emotions. While social media platforms offer unprecedented connectivity and opportunities for self-expression, there is a growing concern about their impact on mental health. The rise of mental health awareness on social media platforms has shed light on the importance of addressing the negative effects of excessive social media use. This blog post will delve into the complex relationship between social media and mental health, exploring both the positive and negative aspects while providing tips for promoting mental well-being in the digital age.

## The Positive and Negative Effects of Social Media on Mental Health
On one hand, social media has the power to connect individuals from across the globe, provide a platform for self-expression, and raise awareness about mental health issues. However, the constant exposure to curated images, unrealistic standards, and cyberbullying on social media can take a toll on one's mental well-being. Recent studies have shown a correlation between excessive social media use and increased feelings of loneliness, anxiety, and depression among young adults. It is crucial to strike a balance between staying connected online and prioritizing one's mental health offline.

## Tips for Managing Social Media Use and Promoting Mental Well-being
To mitigate the negative impact of social media on mental health, it is essential to practice mindfulness and set boundaries when engaging with online platforms. Limiting screen time, unfollowing accounts that trigger negative emotions, and engaging in offline activities that promote well-being are effective strategies for maintaining a healthy relationship with social media. Mental health professionals, social media platforms, and mental health advocates are working together to implement new features that prioritize users' mental well-being, such as digital well-being tools and mental health resources integrated into social media platforms.

## Call to Action
As active users of social media, it is crucial for young adults aged 18-35 to reflect on their social media habits and prioritize their mental health. Seeking professional help when needed, engaging in self-care practices, and fostering offline connections are essential steps in promoting mental well-being in the digital age. By raising awareness about the impact of social media on mental health and taking proactive steps to protect our well-being, we can create a healthier and more balanced relationship with social media.

In conclusion, the intersection of social media and mental health is a complex and evolving landscape that requires thoughtful consideration and proactive measures. By being mindful of our online behaviors, seeking support when needed, and engaging in offline activities that nourish our well-being, we can navigate the digital world with greater resilience and self-awareness. Let us strive to create a digital environment that promotes mental health and fosters positive connections among individuals.

In [15]:
topic = "Motivation or Discipline"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Motivation or Discipline.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unlocking the Power of Motivation and Discipline: A Guide to Achieving Your Goals

Introduction:
In a world filled with distractions and obstacles, finding the motivation and discipline to pursue our goals can be a challenging task. However, by understanding the latest trends, key players, and noteworthy news in the realm of motivation and discipline, we can equip ourselves with the knowledge and tools necessary to overcome these obstacles and achieve success.

Target Audience:
Our target audience in

I now can give a great answer

Final Answer: 

# Unlocking the Power of Motivation and Discipline: A Guide to Achieving Your Goals

## Introduction
In a world filled with distractions and obstacles, finding the motivation and discipline to pursue our goals can be a challenging task. However, by understanding the latest trends, key players, and noteworthy news in the realm of motivation and discipline, we can equip ourselves with the knowledge and tools necessary to overcome these obstacles and achieve success.

## Latest Trends in Motivation and Discipline
Recent studies and research findings have shown that motivation and discipline are crucial factors in achieving goals. Studies have highlighted the importance of intrinsic motivation, which comes from within oneself, as opposed to extrinsic motivation, which comes from external rewards. Additionally, research has shown that discipline, or the ability to stay focused and maintain consistency in one's efforts, is essential for long-ter

In [16]:
Markdown(result)

# Unlocking the Power of Motivation and Discipline: A Guide to Achieving Your Goals

## Introduction
In a world filled with distractions and obstacles, finding the motivation and discipline to pursue our goals can be a challenging task. However, by understanding the latest trends, key players, and noteworthy news in the realm of motivation and discipline, we can equip ourselves with the knowledge and tools necessary to overcome these obstacles and achieve success.

## Latest Trends in Motivation and Discipline
Recent studies and research findings have shown that motivation and discipline are crucial factors in achieving goals. Studies have highlighted the importance of intrinsic motivation, which comes from within oneself, as opposed to extrinsic motivation, which comes from external rewards. Additionally, research has shown that discipline, or the ability to stay focused and maintain consistency in one's efforts, is essential for long-term success.

In the self-improvement industry, popular trends related to motivation and discipline include the rise of mindfulness practices, such as meditation and journaling, as well as the importance of setting SMART goals - specific, measurable, achievable, relevant, and time-bound. These trends emphasize the importance of self-awareness, goal-setting, and self-discipline in achieving personal growth and success.

## Key Players in the Field
Successful individuals who have mastered the art of motivation and discipline serve as inspiration for others looking to achieve their goals. Profiles of individuals like Elon Musk, who is known for his relentless work ethic and dedication to his vision, demonstrate the power of motivation and discipline in achieving extraordinary success. Experts in the fields of psychology and productivity, such as Angela Duckworth and Charles Duhigg, provide valuable insights and tips for cultivating motivation and discipline in one's daily life.

## Noteworthy News
Staying up-to-date on recent news articles and developments related to motivation and discipline can provide valuable insights into how current events can impact our motivation levels. For example, recent studies have shown that remote work and digital distractions can negatively impact one's motivation and productivity. By staying informed about these trends, individuals can take proactive steps to maintain their motivation and discipline in the face of external challenges.

## Developing Motivation and Discipline
Practical strategies for boosting motivation and maintaining discipline are essential for achieving long-term success. Tips such as creating a daily routine, setting specific goals, and rewarding oneself for progress can help individuals stay motivated and focused on their objectives. Additionally, cultivating a growth mindset, which emphasizes learning from failure and embracing challenges, can help individuals maintain their motivation and overcome obstacles on their path to success.

## Call to Action
As we conclude this guide to unlocking the power of motivation and discipline, we encourage readers to implement the strategies discussed in this article. By taking small steps towards cultivating motivation and discipline in their daily lives, individuals can make significant progress towards achieving their goals. For further reading and support, resources such as "The Power of Habit" by Charles Duhigg and "Grit: The Power of Passion and Perseverance" by Angela Duckworth can provide valuable insights and inspiration on the journey to success.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).